## Add Gazebo Model Path
1. Open Bashrc
```
gedit ~/.bashrc
```

2. add the following sentence
```
export GAZEBO_MODEL_PATH=$HOME/tutorial_ws/src/ros_tutorials/gazebo_example/gcamp_gazebo/gazebo_files/models:$GAZEBO_MODEL_PATH
```

## Open Gazebo Launch 

1. Open Terminal
```
roslaunch gcamp_gazebo maze_world.launch
```

2. Wait a few minutes (downloding map data, for the first time)

## Launch Maze Action Server
1. Go  to the simple_action_servers 
```
cd $HOME/your_ws/src/ros_tutorials/actionlib_tutorials/simple_action_servers
```
```
python3 maze_action_server.py
```

## Action Server Code 분석


In [ ]:
import math
import time
import rospy
import actionlib

from mazepkg.basic_cmd_vel import GoForward, Stop, Turn
from mazepkg.gazebo_handler import GazeboResetSimulation
from mazepkg.image_converter import ImageConverter

from nav_msgs.msg import Odometry
from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan
from tf.transformations import euler_from_quaternion

from action_tutorial.msg import MazeAction, MazeFeedback, MazeResult

In [ ]:
class MazeActionClass(object):

    _feedback = MazeFeedback()
    _result = MazeResult()

    def __init__(self, name):
        self._action_name = name
        self._yaw = 0.0
        self._scan = []
        self._current_direction = 3  # 0 1 2 3

        self._cmd_pub = rospy.Publisher("/cmd_vel", Twist, queue_size=1)
        self._odom_sub = rospy.Subscriber("/odom", Odometry, self.odom_callback)
        self._scan_sub = rospy.Subscriber("/scan", LaserScan, self.scan_calback)
        # self._image_sub = rospy.Subscriber('')
        self._action_server = actionlib.SimpleActionServer(
            self._action_name, MazeAction, execute_cb=self.ac_callback, auto_start=False
        )
        self._action_server.start()

        self._rate = rospy.Rate(5)

        print("==== MazeActionClass Constructed ====")
        print("==== Waiting for Client Goal...  ====")

* 외부에서 접근이 가능하도록 publish and subscribe 설정
    * cmd_pub: 로봇에게 이동명령
    * odom_sub: 로봇의 위치 정보
    * scan_sub: 로봇의 라이다 정보 

In [ ]:
int32[] turning_sequence
---
bool success
---
string feedback_msg

In [ ]:
def ac_callback(self, goal):
        success = True
        print("==== Maze Action Server Executing ====")

				# python enumerate
        for i, val in enumerate(goal.turning_sequence):
            # check that preempt has not been requested by the client
            if self._action_server.is_preempt_requested():
                rospy.logwarn("%s: Preempted" % self._action_name)
                self._action_server.set_preempted()
                success = False
                break
						
						# 터미널에 표시될 메세지
            self._feedback.feedback_msg = "Turning to " + direction_str_dict[val]
            self._action_server.publish_feedback(self._feedback)

						# 회전과 전진을 반복합니다.
            print("Turning Sequence : " + str(val))
            self.robot_turn(direction_dict[val])

            self._feedback.feedback_msg = "Moving Forward ..."
            self._action_server.publish_feedback(self._feedback)
            self.robot_go_forward()

            self._rate.sleep()

## Action Client Code 분석


In [ ]:
import time
import rospy
import actionlib

from enum import IntEnum
from action_tutorial.msg import MazeAction, MazeGoal
from tf.transformations import euler_from_quaternion, quaternion_from_euler

# For more detail, search actionlib_msgs/GoalStatus
class ActionState(IntEnum):
    PENDING = 0
    ACTIVE = 1
    PREEMPTED = 2
    SUCCEEDED = 3
    ABORTED = 4
    REJECTED = 5
    PREEMPTING = 6
    RECALLING = 7
    RECALLED = 8
    LOST = 9


def fb_callback(feedback):
    print(feedback)


action_server_name = "/maze_action_server"
rospy.init_node("maze_action_client")
action_client = actionlib.SimpleActionClient(action_server_name, MazeAction)

rospy.loginfo("Action Server Found..." + action_server_name)

goal = MazeGoal()
user_list = []


# try block to handle the exception
try:
    print("Enter numbers [or stop] : ")

    while True:
        user_list.append(int(input()))
# if the input is not-integer, just print the list
except:
    print("Your sequence list : ", user_list)

goal.turning_sequence = user_list

action_client.send_goal(goal, feedback_cb=fb_callback)
state_result = action_client.get_state()

rate = rospy.Rate(1)
rospy.loginfo("State Result from Server : " + str(state_result))

while state_result < ActionState.PREEMPTED:
    # Doing Stuff while waiting for the Server to give a result....
    rate.sleep()
    state_result = action_client.get_state()

if state_result == ActionState.SUCCEEDED:
    rospy.loginfo("Action Done State Result : " + str(state_result))
else:
    rospy.logwarn("Something went wrong, result state : " + str(state_result))